In [47]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import csv, os, pickle
from os.path import isfile, join
from os import listdir
import numpy as np
from table_loader import *
from datetime import datetime,timedelta
from dateutil.parser import parse
#from synapse_parser import *

synapseCacheDir = "/scratch/PI/euan/projects/mhc/data/synapseCache_v2/"
table_path = "/scratch/PI/euan/projects/mhc/data/tables/v2_data_subset/cardiovascular-HealthKitDataCollector-v1.tsv"
subjects = "/scratch/PI/euan/projects/mhc/data/tables/v2_data_subset/subjects/healthkit_data/x0"


def parse_healthkit_steps(file_path):
    tally_dict=dict()
    #read in the data
    dtype_dict=dict()
    dtype_dict['names']=('startTime',
                         'endTime',
                         'type',
                         'value',
                         'unit',
                         'source',
                         'sourceIdentifier')
    dtype_dict['formats']=(datetime,
                           datetime,
                           'S36',
                           'i',
                           'S36',
                           'S36',
                           'S36')
    try:
        data=np.genfromtxt(file_path,
                           dtype=dtype_dict['formats'],
                           names=dtype_dict['names'],
                           delimiter=',',
                           skip_header=True,
                           loose=True,
                           invalid_raise=False,
                           converters={0:lambda x: parse(x),
                                       1:lambda x: parse(x)})
    except:
        return tally_dict
    #get the duration of each activity by day
    try:
        num_rows=len(data)
    except:
        return tally_dict
    for row in range(len(data)):
        try:
            day=data['startTime'][row].date()
            value=data['value'][row]
            datatype=data['sourceIdentifier'][row]
            if day not in tally_dict:
                tally_dict[day]=dict()
            if datatype not in tally_dict[day]:
                tally_dict[day][datatype]=value
            else:
                tally_dict[day][datatype]+=value
        except:
            print "ERROR"
            continue
    return tally_dict


#we assume a subject will rarely have multiple table entries for motion tracker and health kit data in one day,
#but this is possible in the app, so we handle it: 
#sum minute durations together if there is a key conflict for a given day 
def merge_duration_dict(d1,d2):
    #merge duration values 
    d3=dict()
    for entry in d1:
        d3[entry]=d1[entry]
    for entry in d2:
        if entry in d3:
            #sum by key
            for key in d2[entry]:
                if key in d3[entry]:
                    d3[entry][key]+=d2[entry][key]
                else:
                    d3[entry][key]=d2[entry][key]
        else:
            d3[entry]=d2[entry]
    return d3
    #update fraction values

def get_synapse_cache_entry(synapseCacheDir,blob_name):
    #print(str(blob_name)) 
    parent_dir=blob_name[-3::].lstrip('0')
    if parent_dir=="":
        parent_dir="0" 
    mypath=synapseCacheDir+parent_dir+"/"+blob_name
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    for f in onlyfiles:
        if f.startswith('data'):
            return mypath+'/'+f
        
        
outlier_subjects =  ['89a190e2-6b78-46b5-b469-27d231bbf123',
 '441a70b9-9b56-48d5-935a-a7a844abbf57',
 'b5eeefec-7f4a-48c5-aef4-0db7030fb532',
 '105de5d8-e264-47cc-8bdc-d4aff7d568bb',
 '6e070717-16ee-4485-8c05-a6f376b5d1bf',
 '54be93b9-8d7b-4640-b592-956891955dde',
 '6bcb2e5c-cfcf-48c5-bc3a-a0ef78fd4c25',
 '92d1c57e-5cce-4738-89d6-a7f67ed9baf7',
 'df3f739a-35a1-48c5-a64b-a010b952ac35',
 'bcfa41b8-8e9f-48f9-81e1-c27831cadecf']
subject = outlier_subjects[0]
print subject
cur_subject=subject

89a190e2-6b78-46b5-b469-27d231bbf123


In [ ]:
data_table=load_health_kit(table_path)
pickle.dump(data_table,open("cardiovascular-HealthKitDataCollector-v1.p",'wb'))

In [2]:
data_table= pickle.load( open( "cardiovascular-HealthKitDataCollector-v1.p", "rb" ) )
data_subject = data_table[data_table["healthCode"]==subject]
print len(data_subject)

208


In [51]:
subject_distance_vals = {}
for row in data_subject:
    blob_name=row['data']
    if blob_name.endswith('NA'):
        continue 
    synapseCacheFile=get_synapse_cache_entry(synapseCacheDir,blob_name)
    health_kit_distance=parse_healthkit_steps(synapseCacheFile)    
    #continue
    if cur_subject not in subject_distance_vals:
        subject_distance_vals[cur_subject]=health_kit_distance
    else: 
        result = merge_duration_dict(subject_distance_vals[cur_subject],health_kit_distance)
result

ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR


{datetime.date(2017, 1, 3): {'com.garmin.connect.mobile': 174},
 datetime.date(2017, 1, 6): {'com.apple.health.8FDEFD3C-0CB4-49EF-': 11154,
  'com.garmin.connect.mobile': 5712760,
  'edu.stanford.MyHeartCounts': 88},
 datetime.date(2017, 1, 15): {'com.garmin.connect.mobile': 173},
 datetime.date(2017, 1, 17): {'com.apple.health.8FDEFD3C-0CB4-49EF-': 456,
  'com.garmin.connect.mobile': 20076},
 datetime.date(2017, 1, 18): {'com.apple.health.8FDEFD3C-0CB4-49EF-': 1802,
  'com.garmin.connect.mobile': 2351560}}

In [4]:
 blob_name=data_table['data'][row]
            if blob_name.endswith("NA"):
                continue
            synapseCacheFile=get_synapse_cache_entry(synapseCacheDir,blob_name)
            try:
                health_kit_distance=parse_healthkit_steps(synapseCacheFile)
                if cur_subject not in subject_distance_vals:
                    subject_distance_vals[cur_subject]=health_kit_distance
                else: 
                    subject_distance_vals[cur_subject]=merge_duration_dict(subject_distance_vals[cur_subject],health_kit_distance)
            except:
                continue 

1 Exploring MHC Pipeline to find outliers.ipynb
2 Outlier Subject Data.ipynb
aggregators.py
aggregators.pyc
assemble_results_across_subjects.py
assemble_results_across_subjects.sh
cache.p
filter_and_qc.py
filter_and_qc.sh
filters.py
get_activity_fraction_and_duration_appv2.activity_only.sh
get_activity_fraction_and_duration_appv2.healthkit.sh
get_activity_fraction_and_duration_appv2.py
get_objective_metrics_for_gwas.py
get_objective_metrics_for_gwas.sh
health_kit_metrics.py
load_summaries.py
measure_intervention_effects.R
motion_tracker_metrics.py
myheartcounts-master/
outprefix
qc_metrics.py
R/
R test (crashes).ipynb
sbatch.healthkit.sh
sbatch.motiontracker.sh
synapse_parser.py
synapse_parser.pyc
table_loader.py
table_loader.pyc
table_parser.py
table_parser.pyc


In [ ]:
python get_activity_fraction_and_duration_appv2.py --tables /scratch/PI/euan/projects/mhc/data/tables/v2_data_subset/cardiovascular-HealthKitDataCollector-v1.tsv\
       --synapseCacheDir /scratch/PI/euan/projects/mhc/data/synapseCache_v2/\
       --out_prefixes parsed_v2_HealthKitData.$subject_set\
       --data_types health_kit_data_collector\
       --subjects /scratch/PI/euan/projects/mhc/data/tables/v2_data_subset/subjects/healthkit_data/x$subject_set\
       --intervention_metadata /scratch/PI/euan/projects/mhc/data/tables/v2_data_subset/cardiovascular-ABTestResults-v1.tsv


In [12]:
synapseCacheFile=get_synapse_cache_entry(synapseCacheDir,"14337756")
print synapseCacheFile

#read in the data
dtype_dict=dict()
dtype_dict['names']=('startTime',
                     'endTime',
                     'type',
                     'value',
                     'unit',
                     'source',
                     'sourceIdentifier')
dtype_dict['formats']=(datetime,
                       datetime,
                       'S36',
                       'i',
                       'S36',
                       'S36',
                       'S36')
data=np.genfromtxt(synapseCacheFile,
                       dtype=dtype_dict['formats'],
                       names=dtype_dict['names'],
                       delimiter=',',
                       skip_header=True,
                       loose=True,
                       invalid_raise=False,
                       converters={0:lambda x: parse(x),
                                   1:lambda x: parse(x)})

#get the duration of each activity by day


/scratch/PI/euan/projects/mhc/data/synapseCache_v2/756/14337756/data-7f7f8698-be31-494c-a47a-f7676499504b.csv


/opt/rh/python27/root/usr/lib64/python2.7/site-packages/numpy/lib/npyio.py:1641: ConversionWarning: Some errors were detected !
    Line #94 (got 1 columns instead of 7)
    Line #301 (got 1 columns instead of 7)
    Line #337 (got 1 columns instead of 7)
    Line #370 (got 1 columns instead of 7)
  warnings.warn(errmsg, ConversionWarning)


In [13]:
tally_dict=dict()
num_rows=len(data)
num_rows

407

In [27]:
for row in range(len(data)):
    day=data['startTime'][row].date()
    value=data['value'][row]
    datatype=data['type'][row]
        sourceIdentifier=data['sourceIdentifier'][row]
    if day not in tally_dict:
        tally_dict[day]=dict()
    if datatype not in tally_dict[day]:
        tally_dict[day][datatype]=value
    else:
        tally_dict[day][datatype]+=value

    if datatype == "HKQuantityTypeIdentifierStepCount":
        start=data['startTime'][row]
        stop=data['endTime'][row]
        source=data['source'][row]
        sourceIdentifier=data['sourceIdentifier'][row]
        steps_per_second = float(value)/(stop-start).seconds
        if steps_per_second > 4:
            print steps_per_second


In [22]:
#user, average steps, grid of devices giving data

In [28]:
a=set()

In [24]:
a.add()

In [25]:
a.seconds

899

In [33]:
def parse_healthkit_sources(file_path, HK_datatype):
    tally_set=set()
    #read in the data
    dtype_dict=dict()
    dtype_dict['names']=('startTime',
                         'endTime',
                         'type',
                         'value',
                         'unit',
                         'source',
                         'sourceIdentifier')
    dtype_dict['formats']=(datetime,
                           datetime,
                           'S36',
                           'i',
                           'S36',
                           'S36',
                           'S36')
    try:
        data=np.genfromtxt(file_path,
                           dtype=dtype_dict['formats'],
                           names=dtype_dict['names'],
                           delimiter=',',
                           skip_header=True,
                           loose=True,
                           invalid_raise=False,
                           converters={0:lambda x: parse(x),
                                       1:lambda x: parse(x)})
    except:
        return tally_set
    #get the duration of each activity by day
    for row in range(len(data)):
        datatype = data['type'][row]
        sourceIdentifier = data['sourceIdentifier'][row]
        if datatype == HK_datatype:
            tally_set.add(data['sourceIdentifier'][row])
    return tally_set

parse_healthkit_sources(get_synapse_cache_entry(synapseCacheDir,"14337756"),\
                        "HKQuantityTypeIdentifierStepCount")

HELLO


{'com.garmin.connect.mobile'}

In [35]:
def blarg():
    sourses = set()
    for row in data_subject:
        blob_name=row['data']
        if blob_name.endswith('NA'):
            continue 
        synapseCacheFile=get_synapse_cache_entry(synapseCacheDir,blob_name)
        sourses.update(parse_healthkit_sources(synapseCacheFile, "HKQuantityTypeIdentifierStepCount"))
    return sourses
blarg()

14337750 ('616170_634', 'e8a7cefe-045f-483e-a2f2-ae5ada6dd424', 'version 2.0.0, build 3', 'iPhone 6', datetime.datetime(2017, 1, 8, 0, 0), '89a190e2-6b78-46b5-b469-27d231bbf123', 'NA', 'NA', datetime.datetime(2017, 1, 7, 3, 9, 15), 'NA', 'NA', '14337750')
HELLO
14337756 ('616176_634', 'eb93bac0-2b0d-4ea1-a04f-63a9c7f527dc', 'version 2.0.0, build 3', 'iPhone 6', datetime.datetime(2017, 1, 8, 0, 0), '89a190e2-6b78-46b5-b469-27d231bbf123', 'NA', 'NA', datetime.datetime(2017, 1, 7, 3, 4, 4), 'NA', 'NA', '14337756')
HELLO
14337817 ('616204_634', '2db4d8e7-7640-4374-a5a1-f33e46e336ee', 'version 2.0.0, build 3', 'iPhone 6', datetime.datetime(2017, 1, 8, 0, 0), '89a190e2-6b78-46b5-b469-27d231bbf123', 'NA', 'NA', datetime.datetime(2017, 1, 7, 3, 14, 7), 'NA', 'NA', '14337817')
HELLO
14337831 ('616213_634', '80ebfb0f-d255-4bd1-88f9-f05be4a38ee0', 'version 2.0.0, build 3', 'iPhone 6', datetime.datetime(2017, 1, 8, 0, 0), '89a190e2-6b78-46b5-b469-27d231bbf123', 'NA', 'NA', datetime.datetime(2017, 

/opt/rh/python27/root/usr/lib64/python2.7/site-packages/numpy/lib/npyio.py:1641: ConversionWarning: Some errors were detected !
    Line #95 (got 1 columns instead of 7)
    Line #118 (got 1 columns instead of 7)
    Line #129 (got 1 columns instead of 7)
    Line #166 (got 1 columns instead of 7)
    Line #169 (got 1 columns instead of 7)
    Line #172 (got 1 columns instead of 7)
    Line #224 (got 1 columns instead of 7)
    Line #239 (got 1 columns instead of 7)
    Line #316 (got 1 columns instead of 7)
    Line #319 (got 1 columns instead of 7)
    Line #325 (got 1 columns instead of 7)
  warnings.warn(errmsg, ConversionWarning)
/opt/rh/python27/root/usr/lib64/python2.7/site-packages/numpy/lib/npyio.py:1641: ConversionWarning: Some errors were detected !
    Line #89 (got 1 columns instead of 7)
    Line #181 (got 1 columns instead of 7)
    Line #270 (got 1 columns instead of 7)
    Line #275 (got 1 columns instead of 7)
    Line #342 (got 1 columns instead of 7)
    Line #350 (

14337888 ('616245_634', '50b57053-54c5-401d-a7db-f8cc1c9feea2', 'version 2.0.0, build 3', 'iPhone 6', datetime.datetime(2017, 1, 8, 0, 0), '89a190e2-6b78-46b5-b469-27d231bbf123', 'NA', 'NA', datetime.datetime(2017, 1, 7, 3, 8, 9), 'NA', 'NA', '14337888')
HELLO
14337909 ('616252_634', '1e3e911f-55de-47f4-b37c-59bb6dba7da4', 'version 2.0.0, build 3', 'iPhone 6', datetime.datetime(2017, 1, 8, 0, 0), '89a190e2-6b78-46b5-b469-27d231bbf123', 'NA', 'NA', datetime.datetime(2017, 1, 7, 3, 14, 5), 'NA', 'NA', '14337909')
HELLO
14338003 ('616293_634', 'cf84d364-f69c-463c-8dfd-5609f1b98b2f', 'version 2.0.0, build 3', 'iPhone 6', datetime.datetime(2017, 1, 8, 0, 0), '89a190e2-6b78-46b5-b469-27d231bbf123', 'NA', 'NA', datetime.datetime(2017, 1, 7, 3, 8, 23), 'NA', 'NA', '14338003')
HELLO
14338074 ('616339_634', '6b9f2071-5f1b-40b6-9fe8-a28edb9903fd', 'version 2.0.0, build 3', 'iPhone 6', datetime.datetime(2017, 1, 8, 0, 0), '89a190e2-6b78-46b5-b469-27d231bbf123', 'NA', 'NA', datetime.datetime(2017, 

/opt/rh/python27/root/usr/lib64/python2.7/site-packages/numpy/lib/npyio.py:1641: ConversionWarning: Some errors were detected !
    Line #35 (got 1 columns instead of 7)
    Line #61 (got 1 columns instead of 7)
    Line #208 (got 1 columns instead of 7)
  warnings.warn(errmsg, ConversionWarning)


14338086 ('616347_634', '622b8346-3bc8-442d-b0d4-b4ac6cbb4515', 'version 2.0.0, build 3', 'iPhone 6', datetime.datetime(2017, 1, 8, 0, 0), '89a190e2-6b78-46b5-b469-27d231bbf123', 'NA', 'NA', datetime.datetime(2017, 1, 7, 3, 7, 36), 'NA', 'NA', '14338086')
HELLO
14338100 ('616356_634', '2fbafee5-b473-49b3-9780-5ddc45796dd7', 'version 2.0.0, build 3', 'iPhone 6', datetime.datetime(2017, 1, 8, 0, 0), '89a190e2-6b78-46b5-b469-27d231bbf123', 'NA', 'NA', datetime.datetime(2017, 1, 7, 3, 9, 37), 'NA', 'NA', '14338100')
HELLO
14338109 ('616361_634', '33180450-be20-41e8-84a3-075fa7dcdbe7', 'version 2.0.0, build 3', 'iPhone 6', datetime.datetime(2017, 1, 8, 0, 0), '89a190e2-6b78-46b5-b469-27d231bbf123', 'NA', 'NA', datetime.datetime(2017, 1, 7, 3, 9, 7), 'NA', 'NA', '14338109')
HELLO
14338147 ('616380_634', '21a705d8-13c3-4894-bcdf-cefa3308941a', 'version 2.0.0, build 3', 'iPhone 6', datetime.datetime(2017, 1, 8, 0, 0), '89a190e2-6b78-46b5-b469-27d231bbf123', 'NA', 'NA', datetime.datetime(2017, 

...

/opt/rh/python27/root/usr/lib64/python2.7/site-packages/numpy/lib/npyio.py:1641: ConversionWarning: Some errors were detected !
    Line #70 (got 1 columns instead of 7)
    Line #78 (got 2 columns instead of 7)
    Line #83 (got 1 columns instead of 7)
    Line #89 (got 2 columns instead of 7)
    Line #91 (got 1 columns instead of 7)
    Line #101 (got 2 columns instead of 7)
    Line #117 (got 1 columns instead of 7)
    Line #119 (got 1 columns instead of 7)
    Line #121 (got 1 columns instead of 7)
    Line #178 (got 1 columns instead of 7)
    Line #180 (got 1 columns instead of 7)
    Line #182 (got 1 columns instead of 7)
    Line #185 (got 1 columns instead of 7)
    Line #220 (got 1 columns instead of 7)
    Line #263 (got 1 columns instead of 7)
    Line #274 (got 1 columns instead of 7)
    Line #284 (got 1 columns instead of 7)
    Line #290 (got 1 columns instead of 7)
    Line #293 (got 1 columns instead of 7)
    Line #295 (got 1 columns instead of 7)
    Line #299 (go

14398656 ('633902_643', '80d30e76-cf39-45e0-aa39-57ebbf7a448d', 'version 2.0.0, build 3', 'iPhone 6', datetime.datetime(2017, 1, 16, 0, 0), '89a190e2-6b78-46b5-b469-27d231bbf123', 'NA', 'NA', datetime.datetime(2017, 1, 15, 3, 8, 4), 'NA', 'NA', '14398656')
HELLO
14399081 ('634096_643', '3ff30710-08e9-4d57-9b9f-fb5f39f7fabf', 'version 2.0.0, build 3', 'iPhone 6', datetime.datetime(2017, 1, 16, 0, 0), '89a190e2-6b78-46b5-b469-27d231bbf123', 'NA', 'NA', datetime.datetime(2017, 1, 15, 3, 8, 26), 'NA', 'NA', '14399081')
HELLO
14399223 ('634165_643', '91dbbe9b-c7d3-471a-b6ca-2c9b4b7bce0a', 'version 2.0.0, build 3', 'iPhone 6', datetime.datetime(2017, 1, 16, 0, 0), '89a190e2-6b78-46b5-b469-27d231bbf123', 'NA', 'NA', datetime.datetime(2017, 1, 15, 3, 7, 56), 'NA', 'NA', '14399223')
HELLO
14399227 ('634168_643', '7e0814c8-a965-4e4d-9d62-6bfc27c988f7', 'version 2.0.0, build 3', 'iPhone 6', datetime.datetime(2017, 1, 16, 0, 0), '89a190e2-6b78-46b5-b469-27d231bbf123', 'NA', 'NA', datetime.datetime

/opt/rh/python27/root/usr/lib64/python2.7/site-packages/numpy/lib/npyio.py:1641: ConversionWarning: Some errors were detected !
    Line #97 (got 1 columns instead of 7)
  warnings.warn(errmsg, ConversionWarning)


14401697 ('635240_643', '3dde471d-bc90-4a84-ba82-e027f0bbef0a', 'version 2.0.0, build 3', 'iPhone 6', datetime.datetime(2017, 1, 16, 0, 0), '89a190e2-6b78-46b5-b469-27d231bbf123', 'NA', 'NA', datetime.datetime(2017, 1, 15, 3, 8, 28), 'NA', 'NA', '14401697')
HELLO
14401822 ('635304_643', '48555708-fd8c-4a08-8a8e-12265a5e3229', 'version 2.0.0, build 3', 'iPhone 6', datetime.datetime(2017, 1, 16, 0, 0), '89a190e2-6b78-46b5-b469-27d231bbf123', 'NA', 'NA', datetime.datetime(2017, 1, 15, 3, 7, 55), 'NA', 'NA', '14401822')
HELLO
14426357 ('639644_646', '974b561d-ce02-4f66-b2ae-7c0f1fdcc794', 'version 2.0.0, build 3', 'iPhone 6', datetime.datetime(2017, 1, 19, 0, 0), '89a190e2-6b78-46b5-b469-27d231bbf123', 'NA', 'NA', datetime.datetime(2017, 1, 18, 13, 37, 11), 'NA', 'NA', '14426357')
HELLO
14427221 ('640067_646', '67ce8c31-0c73-4509-ae55-d40df1021fdb', 'version 2.0.0, build 3', 'iPhone 6', datetime.datetime(2017, 1, 19, 0, 0), '89a190e2-6b78-46b5-b469-27d231bbf123', 'NA', 'NA', datetime.datet

/opt/rh/python27/root/usr/lib64/python2.7/site-packages/numpy/lib/npyio.py:1641: ConversionWarning: Some errors were detected !
    Line #6 (got 5 columns instead of 7)
    Line #47 (got 1 columns instead of 7)
    Line #49 (got 1 columns instead of 7)
    Line #56 (got 1 columns instead of 7)
    Line #84 (got 1 columns instead of 7)
    Line #91 (got 1 columns instead of 7)
    Line #121 (got 1 columns instead of 7)
    Line #210 (got 1 columns instead of 7)
  warnings.warn(errmsg, ConversionWarning)


In [37]:
sourses

{'com.apple.health.8FDEFD3C-0CB4-49EF-', 'com.garmin.connect.mobile'}